# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 

***

## Import Packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
import os
%matplotlib inline

## Helper Functions

Some functions to be used in lane detection pipeline

In [2]:
def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
            
def draw_lines(image, lines, ymin, ymax, color=[255, 0, 0], thickness=4):
    left_x = np.array([])
    left_y = np.array([])
    right_x = np.array([])
    right_y = np.array([])
    for line in lines:
        for x1, y1, x2, y2 in line:
            if (-2.0 < (y2-y1) / (x2-x1) < -0.5):
                left_x = np.append(left_x, [x1, x2])
                left_y = np.append(left_y, [y1, y2])
            elif  (0.5 < (y2-y1) / (x2-x1) < 2.0):
                right_x = np.append(right_x, [x1, x2])
                right_y = np.append(right_y, [y1, y2])
                
    left_poly = np.poly1d(np.polyfit(left_y, left_x, 1))
    right_poly = np.poly1d(np.polyfit(right_y, right_x, 1))
    
    left_line = [int(left_poly(ymax)), int(ymax), int(left_poly(ymin)), int(ymin)]    
    right_line = [int(right_poly(ymax)), int(ymax), int(right_poly(ymin)), int(ymin)]
    
    cv2.line(image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), color, thickness)
    cv2.line(image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), color, thickness)           

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)    
    return lines

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:    
    initial_img * α + img * β + γ
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Lane Detection Pipeline

In [3]:
def process_image_file(input_image_file, output_image_file):
    input_image = mpimg.imread(input_image_file)
    output_image = process_image(input_image)
    mpimg.imsave(output_image_file, output_image, format="jpg")        
        
def process_image(input_image):
    gray_image = grayscale(input_image)
    
    kernel_size = 3
    blur_image = gaussian_blur(gray_image, kernel_size)

    low_threshold = 100
    high_threshold = 150
    canny_image = canny(blur_image, low_threshold, high_threshold)
       
    imshape = canny_image.shape
    vertices = np.array([[(0, imshape[0]), (imshape[1]/2 - 10, imshape[0]/2 + 50), \
        (imshape[1]/2 + 10, imshape[0]/2 + 50), (imshape[1], imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(canny_image, vertices)

    rho = 1
    theta = np.pi / 180
    threshold = 20
    min_line_len = 30
    max_line_gap = 20
    lines = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    
    lines_image = np.zeros((imshape[0], imshape[1], 3), dtype=np.uint8)
    ymin = imshape[0]/2 + 50
    ymax = imshape[0]
    draw_lines(lines_image, lines, ymin, ymax)

    output_image = weighted_img(lines_image, input_image)
    return output_image

## Test on Images

Run lane detection pipeline on test images

In [4]:
input_dir = "test_images_input/"
output_dir = "test_images_output/"
for image_file in os.listdir(input_dir):
    process_image_file(input_dir + image_file, output_dir + image_file)

## Test on Videos

Run lane detection pipeline on test videos

In [5]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#### Test Clip 1

In [6]:
input_clip_file = "test_videos_input/solidWhiteRight.mp4"
output_clip_file = "test_videos_output/solidWhiteRight.mp4"

input_clip1 = VideoFileClip(input_clip_file)
output_clip = input_clip1.fl_image(process_image)
%time output_clip.write_videofile(output_clip_file, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████▌| 221/222 [00:04<00:00, 47.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 5.09 s


#### Test Clip 2

In [7]:
input_clip_file = "test_videos_input/solidYellowLeft.mp4"
output_clip_file = "test_videos_output/solidYellowLeft.mp4"

input_clip2 = VideoFileClip(input_clip_file)
output_clip = input_clip2.fl_image(process_image)
%time output_clip.write_videofile(output_clip_file, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:14<00:00, 48.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 15.2 s
